<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/Unet%2B%2BV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# CELLA 1: SETUP GLOBALE E MONTAGGIO DRIVE
# =============================================================================
print("--- [FASE 0] Setup Globale ---")

# --- 1. Installazioni ---
!pip install -q segmentation-models-pytorch==0.3.3 albumentations==1.3.1 torchinfo==1.8.0

# --- 2. Import Fondamentali ---
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import shutil
import random
import re
import zipfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
# =============================================================================
# CELLA 2: CONFIGURAZIONE CENTRALE DEL PROGETTO
# =============================================================================
# Modifica i valori in questa cella per controllare l'intera pipeline.

class Config:
    # --- 1. Percorsi Principali su Google Drive ---
    BASE_DRIVE_DIR = Path("/content/drive/MyDrive")
    CHECKPOINT_DIR = BASE_DRIVE_DIR / "PesiUNETPP"

    # Dati per il Pre-Training (le ~600 maschere automatiche)
    PRETRAIN_IMG_DIR = BASE_DRIVE_DIR / "Patches_test/Radio_Patches_Normalized"
    PRETRAIN_MASK_DIR = BASE_DRIVE_DIR / "Patches_test/Patches_masks"

    # Dati "d'oro" per il Fine-Tuning (le 50 maschere corrette a mano)
    GOLDEN_DATA_DIR = BASE_DRIVE_DIR / "GoldenDataset_Clean"

    # Archivio delle patch "d'oro" curate a mano
    GOLDEN_PATCHES_ZIP_DIR = BASE_DRIVE_DIR / "GoldenPatches"
    GOLDEN_PATCHES_ZIP_PATH = GOLDEN_PATCHES_ZIP_DIR / "GoldenPatches_v1.zip"

    # --- 2. Percorsi di Lavoro Locali (su Colab) ---
    # Questi di solito non vanno modificati.
    PRETRAIN_SPLIT_DIR = Path("/content/dataset_radiografico_split")
    FINETUNE_SPLIT_DIR = Path("/content/finetune_full_images_split")
    PATCHES_RAW_DIR = Path("/content/patches_dataset_raw") # Output del generatore di patch
    PATCHES_CURATED_DIR = Path("/content/curated_patches_unzipped") # Patch scompattate

    # --- 3. Nomi dei File dei Pesi ---
    PRETRAINED_MODEL_PATH = CHECKPOINT_DIR / "best_model_pre-training.pth"
    FINETUNED_MODEL_PATH = CHECKPOINT_DIR / "best_model_finetuned.pth"

    # --- 4. Parametri del Modello e Hyperparameters ---
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    ENCODER_NAME = "efficientnet-b0"
    IMG_SIZE = 512
    N_CLASSES = 3

    # Parametri specifici per il PRE-TRAINING
    PRETRAIN_EPOCHS = 25
    PRETRAIN_LR = 1e-3
    PRETRAIN_BATCH_SIZE = 8

    # Parametri specifici per il FINE-TUNING
    FINETUNE_EPOCHS = 20 # Aumentato a 20 per il run finale
    FINETUNE_LR = 1e-5
    FINETUNE_BATCH_SIZE = 4

    # Parametri per la Generazione Patch ("Blackout" Algorithm)
    PATCH_SIZE = 128
    DESIRED_PATCHES_PER_IMAGE = 50 # Genera molti candidati
    BLACKOUT_MARGIN = 90

# --- Creazione automatica delle cartelle necessarie ---
Config.CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
Config.GOLDEN_PATCHES_ZIP_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Configurazione caricata.")
print(f"Dispositivo in uso: {Config.DEVICE}")

In [ ]:
# =============================================================================
#@title CELLA 3: DEFINIZIONI CORE (CLASSI E FUNZIONI)
# =============================================================================

# --- 1. CLASSI DATASET ---

class RadioDataset(Dataset):
    """Dataset generico per caricare immagini (jpg/png) e maschere (png) a 1 canale."""
    def __init__(self, subset_dir, transform=None):
        self.img_dir = subset_dir / "images"
        self.mask_dir = subset_dir / "masks"
        self.transform = transform
        self.mask_paths = sorted(list(self.mask_dir.glob("*.png")))
        self.stems = [p.stem for p in self.mask_paths]
        try:
            self.img_paths = [next(self.img_dir.glob(f"{s}.*")) for s in self.stems]
        except StopIteration:
            # Trova quale file sta causando l'errore per un debug più facile
            for s in self.stems:
                if not list(self.img_dir.glob(f"{s}.*")):
                    raise FileNotFoundError(f"Impossibile trovare img per maschera '{s}.png' in {self.img_dir}")

    def __len__(self): return len(self.stems)

    def __getitem__(self, idx):
        image = cv2.imread(str(self.img_paths[idx]), cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(str(self.mask_paths[idx]), cv2.IMREAD_UNCHANGED)
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image, mask = augmented["image"], augmented["mask"]
        return image, mask.long()

class HybridDataset(Dataset):
    """Dataset che carica un mix di immagini intere e patch curate."""
    def __init__(self, full_img_dir, full_mask_dir, patch_img_dir, patch_mask_dir, transform):
        self.transform = transform
        self.full_mask_paths = sorted(list(full_mask_dir.glob("*.png")))
        self.patch_mask_paths = sorted(list(patch_mask_dir.glob("*.png")))
        self.full_img_paths = [next(full_img_dir.glob(f"{p.stem}.*")) for p in self.full_mask_paths]
        self.patch_img_paths = [next(patch_img_dir.glob(f"{p.stem}.*")) for p in self.patch_mask_paths]
        self.total_size = len(self.full_img_paths) + len(self.patch_img_paths)
        print(f"Dataset ibrido: {len(self.full_img_paths)} immagini intere + {len(self.patch_img_paths)} patch.")

    def __len__(self): return self.total_size

    def __getitem__(self, idx):
        if idx < len(self.full_img_paths):
            img_path, mask_path = self.full_img_paths[idx], self.full_mask_paths[idx]
        else:
            patch_idx = idx - len(self.full_img_paths)
            img_path, mask_path = self.patch_img_paths[patch_idx], self.patch_mask_paths[patch_idx]

        image = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(str(mask_path), cv2.IMREAD_UNCHANGED)
        augmented = self.transform(image=image, mask=mask)
        return augmented["image"], augmented["mask"].long()

# --- 2. FUNZIONI HELPER ---

def extract_hole_id_from_path(path):
    match = re.search(r'H(\d+)', path.stem)
    return f"H{match.group(1)}" if match else None

def copy_files_for_split(stems, subset, source_dir, target_dir):
    """Copia i file corrispondenti agli stems in una sottocartella train/val."""
    (target_dir/subset/"images").mkdir(parents=True, exist_ok=True)
    (target_dir/subset/"masks").mkdir(parents=True, exist_ok=True)
    for stem in stems:
        try:
            img_path = next((source_dir / "images").glob(f"{stem}.*"))
            shutil.copy(img_path, target_dir/subset/"images"/img_path.name)
            shutil.copy(source_dir/"masks"/f"{stem}.png", target_dir/subset/"masks"/f"{stem}.png")
        except StopIteration:
            print(f"ATTENZIONE (split): Immagine non trovata per maschera '{stem}.png'. Saltata.")

print("✅ Definizioni Core caricate (Dataset e Helpers).")

In [ ]:
#@title Cella di creazione maschere 0,1,2
# =============================================================================
# WORKFLOW #0: UTILITY - IL PURIFICATORE DI MASCHERE DA CVAT
#
# SCOPO: Prendere le maschere a colori esportate da CVAT, convertirle nel formato
#        numerico (0,1,2) corretto, e prepararle per essere aggiunte al dataset d'oro.
# INPUT: Cartella con le maschere a colori.
# OUTPUT: Cartella con le maschere numeriche pulite.
# =============================================================================

def run_mask_purification():
    """Converte una cartella di maschere a colori nel formato numerico standard."""
    print("\n" + "="*60 + "\nAVVIO PURIFICAZIONE MASCHERE CVAT\n" + "="*60)

    # --- CONFIGURAZIONE ---
    # La cartella dove hai scompattato le maschere a colori da CVAT
    SOURCE_CVAT_COLOR_MASKS_DIR = Path("/content/drive/MyDrive/CVAT_Exports/SegmentationClass")
    # La cartella dove verranno salvate le maschere pulite
    TARGET_CLEAN_NUMERIC_MASKS_DIR = Path("/content/drive/MyDrive/CVAT_Exports_Cleaned")
    TARGET_CLEAN_NUMERIC_MASKS_DIR.mkdir(parents=True, exist_ok=True)

    if not SOURCE_CVAT_COLOR_MASKS_DIR.exists():
        raise FileNotFoundError(f"Cartella sorgente non trovata: {SOURCE_CVAT_COLOR_MASKS_DIR}")

    # La mappa di conversione (BGR di OpenCV -> Indice di Classe)
    # ATTENZIONE: Questa deve corrispondere ESATTAMENTE ai colori del tuo progetto CVAT.
    COLOR_MAP_BGR = {
        (0, 0, 0): 0,        # background (Nero)
        (100, 100, 255): 1,  # hole (Rosso/Rosa in BGR) - ATTENZIONE AI VALORI
        (220, 220, 0): 2     # damage (Ciano in BGR) - ATTENZIONE AI VALORI
    }

    print(f"Conversione basata sulla mappa BGR: {COLOR_MAP_BGR}")

    # --- CICLO DI CONVERSIONE ---
    mask_files = list(SOURCE_CVAT_COLOR_MASKS_DIR.glob("*.png"))
    print(f"Trovate {len(mask_files)} maschere da convertire...")

    for mask_color_path in tqdm(mask_files, desc="Purificazione maschere"):
        mask_color = cv2.imread(str(mask_color_path), cv2.IMREAD_COLOR)
        if mask_color is None: continue

        h, w, _ = mask_color.shape
        mask_indexed = np.full((h, w), fill_value=0, dtype=np.uint8) # Default a background

        for color_bgr, class_index in COLOR_MAP_BGR.items():
            # Crea una maschera booleana per ogni colore
            matches = np.all(mask_color == color_bgr, axis=-1)
            mask_indexed[matches] = class_index

        save_path = TARGET_CLEAN_NUMERIC_MASKS_DIR / mask_color_path.name
        cv2.imwrite(str(save_path), mask_indexed)

    print(f"\n✅ PURIFICAZIONE COMPLETATA.")
    print(f"Le tue maschere numeriche sono pronte in: '{TARGET_CLEAN_NUMERIC_MASKS_DIR}'")
    print("Ora puoi copiarle manualmente nella cartella 'GoldenDataset_Clean/masks'.")

# Esempio di esecuzione (eseguire solo quando hai nuovi dati da CVAT)
# run_mask_purification()

In [ ]:
# =============================================================================
#@title WORKFLOW #1: PRE-TRAINING DEL MODELLO BASE
#
# SCOPO: Addestrare il modello sulle ~600 maschere automatiche per insegnargli
#        le caratteristiche di base (forma del foro, texture). L'encoder è congelato.
# INPUT: Dati da Config.PRETRAIN_IMG_DIR e Config.PRETRAIN_MASK_DIR.
# OUTPUT: File dei pesi in Config.PRETRAINED_MODEL_PATH.
# =============================================================================

def run_pre_training():
    """
    Esegue l'intero processo di pre-training: preparazione dati, training con encoder
    congelato e salvataggio del modello base.
    """
    print("\n" + "="*60 + "\nINIZIO PIPELINE DI PRE-TRAINING\n" + "="*60)

    # --- 1. Preparazione e Split Dati Grezzi ---
    print("\n--- [Fase 1.1] Preparazione e Split Dati Grezzi ---")
    all_images = list(Config.PRETRAIN_IMG_DIR.glob("**/*.jpg"))
    all_masks = list(Config.PRETRAIN_MASK_DIR.glob("*.png"))

    img_map = {extract_hole_id_from_path(p): p for p in all_images if extract_hole_id_from_path(p) is not None}
    mask_map = {extract_hole_id_from_path(p): p for p in all_masks if extract_hole_id_from_path(p) is not None}

    common_ids = sorted(list(set(img_map.keys()) & set(mask_map.keys())))
    paired_files = [(img_map[id], mask_map[id]) for id in common_ids]

    if Config.PRETRAIN_SPLIT_DIR.exists(): shutil.rmtree(Config.PRETRAIN_SPLIT_DIR)

    # Split 80% train, 10% val, 10% test (il test non è usato qui, ma preparato)
    train_end = int(len(paired_files) * 0.8)
    val_end = train_end + int(len(paired_files) * 0.1)
    train_files, val_files = paired_files[:train_end], paired_files[train_end:val_end]

    def copy_pretrain_files(files, subset_name):
        (Config.PRETRAIN_SPLIT_DIR/subset_name/"images").mkdir(parents=True)
        (Config.PRETRAIN_SPLIT_DIR/subset_name/"masks").mkdir(parents=True)
        for img_path, mask_path in tqdm(files, desc=f"Copia in {subset_name}"):
            shutil.copy(img_path, Config.PRETRAIN_SPLIT_DIR/subset_name/"images"/mask_path.name.replace(".png",".jpg"))
            shutil.copy(mask_path, Config.PRETRAIN_SPLIT_DIR/subset_name/"masks"/mask_path.name)

    copy_pretrain_files(train_files, "train")
    copy_pretrain_files(val_files, "val")
    print(f"✅ Dati di pre-training splittati: {len(train_files)} train, {len(val_files)} val.")

    # --- 2. Dataloaders e Modello ---
    print("\n--- [Fase 1.2] Creazione Dataloaders e Modello ---")
    transform = A.Compose([A.Resize(Config.IMG_SIZE, Config.IMG_SIZE), A.Normalize(mean=(0.5,), std=(0.5,)), ToTensorV2()])
    train_ds = RadioDataset(Config.PRETRAIN_SPLIT_DIR / "train", transform)
    val_ds = RadioDataset(Config.PRETRAIN_SPLIT_DIR / "val", transform)
    train_loader = DataLoader(train_ds, batch_size=Config.PRETRAIN_BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=Config.PRETRAIN_BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    model = smp.UnetPlusPlus(encoder_name=Config.ENCODER_NAME, encoder_weights="imagenet", in_channels=1, classes=Config.N_CLASSES).to(Config.DEVICE)
    for param in model.encoder.parameters(): param.requires_grad = False
    print(f"✅ Modello UNet++ ('{Config.ENCODER_NAME}') con encoder congelato pronto.")

    # --- 3. Loop di Training ---
    print("\n--- [Fase 1.3] Inizio Loop di Training ---")
    loss_fn = smp.losses.DiceLoss(mode='multiclass', from_logits=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=Config.PRETRAIN_LR)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.PRETRAIN_EPOCHS)
    scaler = torch.cuda.amp.GradScaler(enabled=(Config.DEVICE=="cuda"))
    best_val_loss = float('inf')

    for epoch in range(Config.PRETRAIN_EPOCHS):
        model.train()
        for images, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{Config.PRETRAIN_EPOCHS} Train"):
            images, masks = images.to(Config.DEVICE, non_blocking=True), masks.to(Config.DEVICE, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=(Config.DEVICE=="cuda")):
                outputs = model(images)
                loss = loss_fn(outputs, masks)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{Config.PRETRAIN_EPOCHS} Val"):
                images, masks = images.to(Config.DEVICE, non_blocking=True), masks.to(Config.DEVICE, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(Config.DEVICE=="cuda")):
                    val_loss += loss_fn(model(images), masks).item()

        avg_val_loss = val_loss / max(1, len(val_loader))
        print(f"Pre-training Epoch {epoch+1}: Val Loss: {avg_val_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), Config.PRETRAINED_MODEL_PATH)
            print(f"🏆 Checkpoint pre-training salvato (Val Loss: {avg_val_loss:.4f}).")

        scheduler.step()

    print("\n🎉 Pre-addestramento Finito.")
    print(f"Modello base salvato in: {Config.PRETRAINED_MODEL_PATH}")

# Esempio di esecuzione (da non eseguire se non necessario)
# run_pre_training()

##Celle di craezione patches


In [ ]:
# =============================================================================
# WORKFLOW #2.A: GENERATORE DI PATCH "OCCHIO DI FALCO"
#
# SCOPO: Isolare i dettagli complessi del danno (ragnatele) creando patch mirate.
#        Usa l'algoritmo "Blackout" per escludere il foro.
# INPUT: Dati da Config.GOLDEN_DATA_DIR.
# OUTPUT: Cartella locale '/content/patches_dataset_raw' piena di candidati.
# =============================================================================

def generate_patches():
    """Usa l'algoritmo 'Blackout' per generare le patch di dettagli."""
    print("\n" + "="*60 + "\nINIZIO GENERAZIONE PATCH\n" + "="*60)

    # Pulizia delle cartelle di lavoro
    if Path('/content/temp_finetune_split').exists(): shutil.rmtree('/content/temp_finetune_split')
    if Config.PATCHES_RAW_DIR.exists(): shutil.rmtree(Config.PATCHES_RAW_DIR)

    # Split temporaneo dei dati d'oro per processarli
    all_stems = sorted([p.stem for p in (Config.GOLDEN_DATA_DIR / "masks").glob("*.png")])
    train_stems, val_stems = train_test_split(all_stems, test_size=0.2, random_state=42)
    copy_files_for_split(train_stems, "train", Config.GOLDEN_DATA_DIR, Path('/content/temp_finetune_split'))
    copy_files_for_split(val_stems, "val", Config.GOLDEN_DATA_DIR, Path('/content/temp_finetune_split'))
    print("✅ Dati d'oro splittati in locale per il processamento.")

    for subset in ["train", "val"]:
        print(f"\nProcesso la subset: {subset}...")
        source_img_dir = Path('/content/temp_finetune_split')/subset/"images"
        source_mask_dir = Path('/content/temp_finetune_split')/subset/"masks"
        target_img_dir = Config.PATCHES_RAW_DIR/subset/"images"
        target_mask_dir = Config.PATCHES_RAW_DIR/subset/"masks"
        target_img_dir.mkdir(parents=True)
        target_mask_dir.mkdir(parents=True)
        stems = [p.stem for p in source_mask_dir.glob("*.png")]

        for stem in tqdm(stems, desc=f"Applicando blackout su '{subset}'"):
            img_path = next(source_img_dir.glob(f"{stem}.*"))
            mask_path = source_mask_dir / f"{stem}.png"
            img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
            mask = cv2.imread(str(mask_path), cv2.IMREAD_UNCHANGED)
            if img is None: continue

            hole_mask = np.uint8(mask == 1) # Assumendo 1=hole, 2=damage. Cambiare se necessario
            damage_mask = np.uint8(mask == 2)

            # --- Logica Blackout ---
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (Config.BLACKOUT_MARGIN*2, Config.BLACKOUT_MARGIN*2))
            blackout_zone = cv2.dilate(hole_mask, kernel, iterations=1)
            surviving_damage_mask = damage_mask.copy()
            surviving_damage_mask[blackout_zone > 0] = 0

            surviving_points_yx = np.argwhere(surviving_damage_mask > 0)
            if len(surviving_points_yx) == 0: continue

            num_to_sample = min(Config.DESIRED_PATCHES_PER_IMAGE, len(surviving_points_yx))
            sampled_indices = np.random.choice(len(surviving_points_yx), size=num_to_sample, replace=False)

            for i, idx in enumerate(sampled_indices):
                y, x = surviving_points_yx[idx]
                y_min=max(0,y-Config.PATCH_SIZE//2); x_min=max(0,x-Config.PATCH_SIZE//2)
                y_max=y_min+Config.PATCH_SIZE; x_max=x_min+Config.PATCH_SIZE
                if y_max>img.shape[0] or x_max>img.shape[1]: continue

                img_patch = img[y_min:y_max, x_min:x_max]
                mask_patch = mask[y_min:y_max, x_min:x_max]

                if img_patch.shape == (Config.PATCH_SIZE, Config.PATCH_SIZE):
                    cv2.imwrite(str(target_img_dir/f"{stem}_patch_{i}.png"), img_patch)
                    cv2.imwrite(str(target_mask_dir/f"{stem}_patch_{i}.png"), mask_patch)

    shutil.rmtree('/content/temp_finetune_split')
    print(f"\n🎉 Generazione Patch Completata. I candidati sono in '{Config.PATCHES_RAW_DIR}'")

# Esempio di esecuzione (da non eseguire se non necessario)
# generate_patches()

In [ ]:
# =============================================================================
# WORKFLOW #2.B: CURA MANUALE E ARCHIVIAZIONE PATCH
#
# SCOPO: Selezionare a mano le patch migliori dalla cartella di candidati
#        e impacchettarle in un singolo file ZIP su Google Drive per uso futuro.
# INPUT: Cartella '/content/patches_dataset_raw' e una lista di nomi di file.
# OUTPUT: File ZIP in Config.GOLDEN_PATCHES_ZIP_PATH.
# =============================================================================

def curate_and_zip_patches(list_of_best_stems):
    """
    Prende una lista di nomi di file di patch, li recupera, e li archivia in uno ZIP.
    """
    print("\n" + "="*60 + "\nINIZIO CURA E ARCHIVIAZIONE PATCH\n" + "="*60)

    TEMP_CURATED_DIR = Path("/content/curated_patches_for_zip")
    if TEMP_CURATED_DIR.exists(): shutil.rmtree(TEMP_CURATED_DIR)

    target_img_dir = TEMP_CURATED_DIR / "images"
    target_mask_dir = TEMP_CURATED_DIR / "masks"
    target_img_dir.mkdir(parents=True)
    target_mask_dir.mkdir(parents=True)

    found_count = 0
    for stem in tqdm(list_of_best_stems, desc="Recuperando i file scelti"):
        found = False
        for subset in ["train", "val"]:
            source_img_path = Config.PATCHES_RAW_DIR / subset / "images" / f"{stem}.png"
            source_mask_path = Config.PATCHES_RAW_DIR / subset / "masks" / f"{stem}.png"
            if source_img_path.exists():
                shutil.copy(source_img_path, target_img_dir / source_img_path.name)
                shutil.copy(source_mask_path, target_mask_dir / source_mask_path.name)
                found = True
                break
        if found:
            found_count += 1

    print(f"\nRecuperati con successo {found_count} su {len(list_of_best_stems)} file richiesti.")

    if found_count > 0:
        # Crea il nome del file con una versione (es. v1, v2)
        zip_name = Config.GOLDEN_PATCHES_ZIP_PATH.stem.split('_v')[0] # Prende 'GoldenPatches'
        version = 1
        while (Config.GOLDEN_PATCHES_ZIP_DIR / f"{zip_name}_v{version}.zip").exists():
            version += 1

        final_zip_path = Config.GOLDEN_PATCHES_ZIP_DIR / f"{zip_name}_v{version}.zip"

        # Crea l'archivio
        shutil.make_archive(str(final_zip_path.with_suffix('')), 'zip', TEMP_CURATED_DIR)
        shutil.rmtree(TEMP_CURATED_DIR)

        print(f"\n🎉 ARCHIVIO CREATO: '{final_zip_path}'")
    else:
        print("\nNessun file recuperato. Controlla i nomi nella lista.")

# Esempio di esecuzione:
BEST_PATCH_STEMS = [ "H569_..._patch_3", "H554_..._patch_11" ]
curate_and_zip_patches(BEST_PATCH_STEMS)

##fine creazione patches


#PArte 2


In [ ]:
# =============================================================================
# WORKFLOW #3: FINE-TUNING IBRIDO "OCCHIO DI FALCO"
#
# SCOPO: Addestrare il modello finale usando un mix di dati "d'oro" interi
#        (per il contesto) e le patch curate (per i dettagli).
#        Tutti i layer del modello sono sbloccati e il learning rate è basso.
# INPUT: Checkpoint dal pre-training (o precedente fine-tuning) e ZIP delle patch curate.
# OUTPUT: File dei pesi finale in Config.FINETUNED_MODEL_PATH.
# =============================================================================

def run_fine_tuning():
    """
    Esegue il training ibrido, caricando le immagini intere e le patch curate.
    """
    print("\n" + "="*60 + "\nINIZIO PIPELINE DI FINE-TUNING IBRIDO\n" + "="*60)

    # --- 1. Preparazione Dati ---
    print("\n--- [Fase 3.1] Preparazione Dati per il Fine-Tuning ---")
    if Config.FINETUNE_SPLIT_DIR.exists(): shutil.rmtree(Config.FINETUNE_SPLIT_DIR)
    if Config.PATCHES_CURATED_DIR.exists(): shutil.rmtree(Config.PATCHES_CURATED_DIR)

    if not Config.GOLDEN_PATCHES_ZIP_PATH.exists():
      raise FileNotFoundError(f"Archivio ZIP delle patch non trovato in {Config.GOLDEN_PATCHES_ZIP_PATH}. Esegui prima la cura delle patch.")

    with zipfile.ZipFile(Config.GOLDEN_PATCHES_ZIP_PATH, 'r') as zf:
        zf.extractall(Config.PATCHES_CURATED_DIR)
    print(f"✅ Patch d'oro estratte in '{Config.PATCHES_CURATED_DIR}'")

    # Split 80/20 delle immagini "d'oro" INTERE
    all_full_stems = sorted([p.stem for p in (Config.GOLDEN_DATA_DIR / "masks").glob("*.png")])
    train_stems, val_stems = train_test_split(all_full_stems, test_size=0.2, random_state=42)
    copy_files_for_split(train_stems, "train", Config.GOLDEN_DATA_DIR, Config.FINETUNE_SPLIT_DIR)
    copy_files_for_split(val_stems, "val", Config.GOLDEN_DATA_DIR, Config.FINETUNE_SPLIT_DIR)
    print(f"✅ Immagini d'oro intere splittate: {len(train_stems)} train, {len(val_stems)} val.")

    # --- 2. Dataloaders Ibridi ---
    print("\n--- [Fase 3.2] Creazione Dataloaders Ibridi ---")
    transform = A.Compose([A.Resize(Config.IMG_SIZE, Config.IMG_SIZE), A.HorizontalFlip(), A.VerticalFlip(), A.Normalize(mean=(0.5,), std=(0.5,)), ToTensorV2()])

    train_ds = HybridDataset(
        Config.FINETUNE_SPLIT_DIR/"train"/"images", Config.FINETUNE_SPLIT_DIR/"train"/"masks",
        Config.PATCHES_CURATED_DIR/"images", Config.PATCHES_CURATED_DIR/"masks",
        transform
    )
    # Nota: Usiamo un approccio semplificato dove il val set ibrido contiene anche le patch di training
    # Per una valutazione purista, le patch dovrebbero essere splittate a loro volta.
    val_ds = HybridDataset(
        Config.FINETUNE_SPLIT_DIR/"val"/"images", Config.FINETUNE_SPLIT_DIR/"val"/"masks",
        Config.PATCHES_CURATED_DIR/"images", Config.PATCHES_CURATED_DIR/"masks",
        transform
    )

    train_loader = DataLoader(train_ds, batch_size=Config.FINETUNE_BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=Config.FINETUNE_BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    # --- 3. Modello e Loop di Training ---
    print("\n--- [Fase 3.3] Inizio Loop di Fine-Tuning ---")
    model = smp.UnetPlusPlus(encoder_name=Config.ENCODER_NAME, in_channels=1, classes=Config.N_CLASSES).to(Config.DEVICE)

    # Parte dall'ultimo fine-tuning se esiste, altrimenti dal pre-training
    model_to_load = Config.FINETUNED_MODEL_PATH if Config.FINETUNED_MODEL_PATH.exists() else Config.PRETRAINED_MODEL_PATH
    print(f"Caricamento pesi da: {model_to_load}")
    model.load_state_dict(torch.load(model_to_load, map_location=Config.DEVICE))

    # Sblocca tutti i parametri per il fine-tuning
    for p in model.parameters(): p.requires_grad = True
    print("✅ Modello sbloccato per il fine-tuning.")

    loss_fn = smp.losses.DiceLoss(mode='multiclass', from_logits=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=Config.FINETUNE_LR)
    scaler = torch.cuda.amp.GradScaler(enabled=(Config.DEVICE=="cuda"))
    best_val_loss = float('inf')

    for epoch in range(Config.FINETUNE_EPOCHS):
        model.train()
        for images, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{Config.FINETUNE_EPOCHS} Train"):
            images, masks = images.to(Config.DEVICE, non_blocking=True), masks.to(Config.DEVICE, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=(Config.DEVICE=="cuda")):
                outputs = model(images)
                loss = loss_fn(outputs, masks)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{Config.FINETUNE_EPOCHS} Val"):
                images, masks = images.to(Config.DEVICE, non_blocking=True), masks.to(Config.DEVICE, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(Config.DEVICE=="cuda")):
                    val_loss += loss_fn(model(images), masks).item()

        avg_val_loss = val_loss / max(1, len(val_loader))
        print(f"Fine-Tuning Epoch {epoch+1}: Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), Config.FINETUNED_MODEL_PATH)
            # Salviamo una copia locale per scaricarla subito se serve
            shutil.copy(Config.FINETUNED_MODEL_PATH, "/content/best_model_finetuned_local.pth")
            print(f"🏆 Checkpoint fine-tuning salvato (Val Loss: {avg_val_loss:.4f}).")

    print("\n🎉 Fine-Tuning Ibrido Terminato.")

# Esempio di esecuzione (da non eseguire se non necessario)
# run_fine_tuning()

In [ ]:
# =============================================================================
# WORKFLOW #4: VALUTAZIONE FINALE E CONFRONTO VISIVO
#
# SCOPO: Confrontare visivamente le performance del modello pre-training
#        contro il modello finale post-fine-tuning ibrido.
# INPUT: Pesi da Config.PRETRAINED_MODEL_PATH e Config.FINETUNED_MODEL_PATH.
#        Dati dal validation set del tuo dataset d'oro.
# OUTPUT: Plot di confronto a 4 pannelli.
# =============================================================================

def run_evaluation():
    """
    Esegue il confronto visivo finale tra il modello di pre-training e quello
    dopo il fine-tuning ibrido, usando il validation set dei dati d'oro.
    """
    print("\n" + "="*60 + "\nINIZIO PIPELINE DI VALUTAZIONE COMPARATIVA\n" + "="*60)

    # --- 1. Preparazione Dati e Modelli ---
    print("\n--- [Fase 4.1] Preparazione dati e caricamento modelli ---")
    transform = A.Compose([A.Resize(Config.IMG_SIZE, Config.IMG_SIZE), A.Normalize(mean=(0.5,), std=(0.5,)), ToTensorV2()])

    # Usa il validation set delle immagini INTERE per un confronto equo "nel mondo reale"
    val_dataset_full = RadioDataset(Config.FINETUNE_SPLIT_DIR / "val", transform=transform)
    print(f"Valutazione su {len(val_dataset_full)} campioni del validation set 'd'oro'.")

    # Carica entrambi i modelli
    model_before = smp.UnetPlusPlus(encoder_name=Config.ENCODER_NAME, in_channels=1, classes=Config.N_CLASSES).to(Config.DEVICE)
    model_after = smp.UnetPlusPlus(encoder_name=Config.ENCODER_NAME, in_channels=1, classes=Config.N_CLASSES).to(Config.DEVICE)

    if not Config.PRETRAINED_MODEL_PATH.exists() or not Config.FINETUNED_MODEL_PATH.exists():
        raise FileNotFoundError("Uno o entrambi i file dei pesi (pre-trained e fine-tuned) non sono stati trovati.")

    model_before.load_state_dict(torch.load(Config.PRETRAINED_MODEL_PATH, map_location=Config.DEVICE)); model_before.eval()
    model_after.load_state_dict(torch.load(Config.FINETUNED_MODEL_PATH, map_location=Config.DEVICE)); model_after.eval()
    print("✅ Modelli PRE e POST caricati.")

    # --- 2. Esecuzione del Confronto Visivo ---
    print("\n--- [Fase 4.2] Generazione del confronto visivo ---")
    with torch.no_grad():
        for i in range(len(val_dataset_full)):
            image_tensor, mask_gt_tensor = val_dataset_full[i]
            input_tensor = image_tensor.unsqueeze(0).to(Config.DEVICE)

            # Inferenza con entrambi i modelli
            output_before = model_before(input_tensor)
            pred_before = torch.argmax(output_before.squeeze(), dim=0).cpu().numpy()

            output_after = model_after(input_tensor)
            pred_after = torch.argmax(output_after.squeeze(), dim=0).cpu().numpy()

            # De-normalizza l'immagine per una visualizzazione corretta
            image_vis = image_tensor.permute(1, 2, 0).numpy() * 0.5 + 0.5
            stem = val_dataset_full.stems[i]

            fig, axes = plt.subplots(1, 4, figsize=(28, 7))
            fig.suptitle(f'Confronto per: {stem}', fontsize=16, y=0.95)
            axes[0].imshow(image_vis, cmap='gray'); axes[0].set_title("Immagine Originale")
            axes[1].imshow(mask_gt_tensor.numpy(), cmap='viridis', vmin=0, vmax=2); axes[1].set_title("Ground Truth")
            axes[2].imshow(pred_before, cmap='viridis', vmin=0, vmax=2); axes[2].set_title("PRE-TRAINING")
            axes[3].imshow(pred_after, cmap='viridis', vmin=0, vmax=2); axes[3].set_title("POST-IBRIDO")
            for ax in axes: ax.axis('off')
            plt.show()

    print("\n🎉 Valutazione Completata.")

# Esempio di esecuzione (da non eseguire se non necessario)
# run_evaluation()